In [2]:
# !pip install tensorflow mlflow matplotlib
# !pip install --upgrade sagemaker
# !pip install keras-tuner


In [3]:
import tensorflow as tf
import sagemaker

sess    = sagemaker.Session()
role    = sagemaker.get_execution_role()
# account = sess.boto_session.client('sts').get_caller_identity()['Account']
# region  = sess.boto_session.region_name

In [4]:
import os
import keras
import numpy as np
from tensorflow.keras.datasets import mnist
(x_train, y_train), (x_val, y_val) = mnist.load_data()

os.makedirs("./data", exist_ok = True)

np.savez('./data/training', image=x_train, label=y_train)
np.savez('./data/validation', image=x_val, label=y_val)

In [5]:
prefix = 'keras-mnist'

training_input_path   = sess.upload_data('data/training.npz', key_prefix=prefix+'/training')
validation_input_path = sess.upload_data('data/validation.npz', key_prefix=prefix+'/validation')

print(training_input_path)
print(validation_input_path)

s3://sagemaker-us-east-1-992774543208/keras-mnist/training/training.npz
s3://sagemaker-us-east-1-992774543208/keras-mnist/validation/validation.npz


In [6]:
from sagemaker.tensorflow import TensorFlow

tf_estimator = TensorFlow(entry_point='train.py', 
                          role=role,
                          train_instance_count=1, 
                          train_instance_type='local',
                          framework_version='1.15', 
                          py_version='py3',
                          script_mode=True
                         )

estimator = TensorFlow(
    entry_point="cifar10_keras_main.py",
    source_dir="source_dir",
    role=role,
    framework_version="1.15.2",
    py_version="py3",
    hyperparameters=local_hyperparameters,
    train_instance_count=1,
    train_instance_type=instance_type,
)

train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [7]:
tf_estimator